In [1]:
import import_ipynb
from RC_Transformer import transformer, CustomSchedule
from CommonModule.Handle_Dir import mkdir_p, del_folder
from CommonModule.Encoder import IntegerEncoder
from CommonModule.Decoder import Decoder

import os
import re
import csv
import pandas as pd
import numpy as np
from glob import iglob
import tensorflow as tf
from glove import Corpus, Glove


importing Jupyter notebook from RC_Transformer.ipynb
importing Jupyter notebook from /data/ksb/TestSampleDir/articleSummary-Jupyter/Transformer/CommonModule/Handle_Dir.ipynb
importing Jupyter notebook from /data/ksb/TestSampleDir/articleSummary-Jupyter/Transformer/CommonModule/Encoder.ipynb
importing Jupyter notebook from /data/ksb/TestSampleDir/articleSummary-Jupyter/Transformer/CommonModule/Decoder.ipynb


In [2]:
BASE_DIR = "/data/ksb/TestSampleDir"
DATA_BASE_DIR = os.path.join(BASE_DIR, 'articles')

PREPROCESSED_PATH = os.path.join(DATA_BASE_DIR,"Preprocessed-Data")
SUMMARY_PREPROCESSED_PATH = os.path.join(DATA_BASE_DIR,"Summary-Preprocessed-Data")
PREDICT_PATH = os.path.join(DATA_BASE_DIR,"Predict-Data")

WORD_ENCODING_DIR = os.path.join(os.path.join(BASE_DIR, 'articleSummary-Jupyter'), 'Word-Encoding-Model')
MODEL_DIR = os.path.join(os.path.join(BASE_DIR, 'articleSummary-Jupyter'), 'Transformer-Model')


In [3]:
D_MODEL = 128
MIN_COUNT = 10
VOCAB_SIZE = 70000
SUMMARY_MAX_LEN = 100

In [4]:
import sentencepiece as spm
sp = spm.SentencePieceProcessor()
model_num = len(list(iglob(os.path.join(WORD_ENCODING_DIR, 'spm-summary-*.vocab'), recursive=False))) -1
sp.Load(os.path.join(WORD_ENCODING_DIR, 'spm-summary-{}.model').format(model_num))

True

In [5]:
START_TOKEN = [sp.bos_id()]
END_TOKEN = [sp.eos_id()]

LAYER_NUM = 6
NUM_HEADS = 8
DFF = 512

BATCH_SIZE = 64
BUFFER_SIZE = 5000

WARMUP_STEPS = 50
EPOCHS = 70

Transformer 모델 생성

In [6]:
with tf.device('/CPU:0'):
    model = transformer(
        vocab_size=VOCAB_SIZE,
        num_layers=LAYER_NUM,
        dff=DFF,
        d_model=D_MODEL,
        num_heads=NUM_HEADS,
        dropout = 0.3)

(None, None, 128)
(None, None, 128)


학습된 가중치 로드

In [7]:
checkpoint_path = os.path.join(MODEL_DIR, 'RC_Transformer')
print(checkpoint_path)

model.load_weights(checkpoint_path)

/data/ksb/TestSampleDir/articleSummary-Jupyter/Transformer-Model/RC_Transformer


In [8]:
options = {
        'model-type' : 'Sentence-Piece',
        'inv_wv' : None,
        'corpus' : None,
        'spm' : sp
    }

In [15]:
def evaluate(content, max_len):
    
    enc_input_sent = IntegerEncoder(options=options, filepaths=None).line_encoder(content)
    enc_input_sent = START_TOKEN + enc_input_sent + END_TOKEN
    enc_input_sent = tf.expand_dims(enc_input_sent, axis=0)
    
    output = START_TOKEN
    output = tf.expand_dims(output, axis=0)
            
    for i in range(max_len):
        predictions = model(inputs=[enc_input_sent, output], training=False)
        
        predictions = predictions[:, -1:, :]
        
        predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)
        predicted_id = tf.squeeze(predicted_id)
#         print("index : {idx}, predicted : {pre}".format(idx=i, pre=predicted_id))

        if tf.equal(predicted_id, END_TOKEN[0]):
            break

        output = tf.concat([output, tf.expand_dims([predicted_id], axis=0)], axis=-1)

    return tf.squeeze(output, axis=0)


In [10]:
def predict(content):
    prediction = evaluate(content, SUMMARY_MAX_LEN)
    options = {
        'model-type' : 'Sentence-Piece',
        'inv_wv' : None,
        'corpus' : None,
        'spm' : sp
    }
    predict_list = prediction.numpy().tolist()
    decoder = Decoder(options)
    predicted_sentence = decoder.decode(predict_list)

    print('Input: {}'.format(content))
    print('Output: {}'.format(' '.join(predicted_sentence)))

    return predicted_sentence

In [11]:
def saveCSVFile(baseDir, media, article_dist):
    save_path = os.path.join(baseDir, media) + ".csv"

    article_dist.to_csv(save_path, mode='w', header=False)

In [12]:
def get_media_name(filepath):
    filename = filepath.split(os.sep)[-1]
    return filename.split(".")[0]

In [17]:
del_folder(PREDICT_PATH)
mkdir_p(PREDICT_PATH)

In [18]:
generated_summary_dist = pd.DataFrame(columns=['Title', 'Origin Contents', 'Generated Summary'])

for _, proc_path in enumerate(iglob(os.path.join(PREPROCESSED_PATH, '**.csv'), recursive=False)):
    
    media_name = get_media_name(proc_path)
    summary_path = os.path.join(SUMMARY_PREPROCESSED_PATH, media_name +".csv")
    print(media_name, proc_path)
    
    f_src = open(proc_path, 'r', newline="\n", encoding="utf-8")
    f_tar = open(summary_path, 'r', newline="\n", encoding="utf-8")
    
    for [_, title, contents], [_, _, target] in zip(csv.reader(f_src), csv.reader(f_tar)):
        
        content = contents.split("\t")
        target_summary = target.split("\t")
                
        content_line = ' '.join(content)
        predict_summary = ' '.join(predict(content_line))
        target_summary = ' '.join(target_summary)
        print(predict_summary)

        summary = {'Title' : title, 'Target Summary' : target_summary, 'Generated Summary' : predict_summary}
        generated_summary_dist = generated_summary_dist.append(summary, ignore_index=True)
    
    saveCSVFile(PREDICT_PATH, media_name, generated_summary_dist)
    f.close()

스포츠동아 /data/ksb/TestSampleDir/articles/Preprocessed-Data/스포츠동아.csv
Input: 8일 방송한 MBC 실화탐사대 어머니의 시신을 방치한 채 살아가는 남자와 고등학생에게 성 착취 등의 협박을 당해온 13세 소녀의 안타까운 사연으로 눈길을 끌었다 제작진은 노모가 사망한 후 약 한 달이 지나도록 장례를 치르지 않고 있다는 한 남성을 찾았다 그는 어머니가 돌아가신 후 여자 원피스를 입고 돌아다니거나 인근 식당에서 무전취식을 하는 기이한 행동을 일삼았다 집안에 온갖 쓰레기를 모아둔 탓에 악취와 벌레가 들끓어 이웃들의 피해가 이만저만 아니었다 지적 장애를 가진 그의 행동에는 나름의 이유가 있었다 그는 집안을 가득 채운 쓰레기들이 대부분 엄마의 물건 이라며 버리고 싶지 않다고 밝혔다 그가 입고 다니던 원피스 역시 엄마의 옷이었다 아들은 엄마 생각나서 엄마 보고 싶어서 라며 돌아가신 엄마의 흔적을 차마 지우지 못하는 속내를 드러냈다 장례를 치르지 못한 이유는 비용을 지불할 능력이 없어서라고 털어놨다 그가 지적 장애를 가졌음에도 도움을 받을 수 없는 이유는 장애 등급 때문이었다 주민 센터 관계자는 장애인 연금 신청을 하려면 중증이어야 하는데 분은 신청할 수 있는 등급이 라고 설명했다 복지 사각지대에 놓인 그는 어머니에게 배운 대로 교회 무료 급식소 등을 전전하며 가끔 식사를 제공해주는 이웃들의 도움에 기대고 있었다 안타까운 사정에 처한 그를 위해 다행히 관계 기관이 나섰다 덕분에 어머니의 장례는 무사히 치러졌다 관할 구청과 주민 센터 역시 그의 자립을 돕겠다고 약속했다 주거 환경 개선을 위해 청소 업체도 나섰다 이성규 한국장애인재단 이사장은 발달 장애 3급이면 일상생활이 가능하도록 반복적으로 교육하고 적응 훈련을 시키면 될 며 홀로서기가 불가능한 일이 아님을 밝혔다 실화탐사대 는 지속적인 관리 감독이 가능한 제도 보완의 필요성을 강조하며 그의 앞날을 응원했다 이날 실화탐사대 두 번째 사연은 고등학생에게 온갖 협박을 당한 13살

Input: 여의도 칼부림 흉기 난동 극단적인 선택여의도 칼부림 사건이 발생했다 12일 오전 9시15분께 서울 영등포구 여의도의 한 건물의 지하 식당에서 50대 여성 A씨가 30대 남성 B씨를 흉기로 찌르는 사건이 일어났다 이날 서울 영등포경찰서와 소방당국 등에 따르면 A씨는 여의도 식당에서 일했던 B씨를 흉기로 찌른 뒤 스스로 극단적인 선택을 한 것으로 알려졌다 A씨는 칼부림 후 현장에서 숨진 채 발견됐다 피해 남성 B씨는 복부에 자산을 입은 채 인근 병원으로 옮겨졌다 경찰은 여의도 칼부림 사건의 범행 동기 정확한 사건 경위를 조사하고 동아닷컴
Output: 앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커
앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵

Input: 다양한 할로윈 포토존 해저왕국 콘셉트 어트랙션 매직아일랜드 캐슬 좀비 M 공연 하루 2회 진행 테마파크가 크리스마스와 하반기 양대 중요 이벤트 시즌으로 꼽는 할로윈이 다가왔다 서울 잠실 롯데월드 어드벤처도 가족 친구 연인 다양한 방문객들을 만족시킬 즐길거리를 마련했다 실내 어드벤처 지역은 큐티한 할로윈으로 변신했다 눈길을 끄는 친근하고 귀여운 호박유령 포토존과 할로윈 빅로티 포토존을 마련했다 해저 왕국 테마의 새 어트랙션도 스릴 어트랙션이 많은 야외 매직아일랜드에 새로 오프한 머킹의 회전목마 는 돌고래 해마 물고기 귀여운 바다 생물로 구성했다 할로윈 공연으로는 로티 로리와 큐티 할로윈 캐릭터들이 총 출동하는 로티스 할로윈 파티 퍼레이드 오후 6시30분에는 메인 무대인 가든 스테이지에서 뮤지컬 쇼 드라큐라의 사랑 무대가 펼쳐진다 16일 재오픈한 셀프 포토 스튜디오 그럴싸진관 도 할로윈 마녀의 숲 부스 가을 캠핑 부스 가을에 맞는 컬러와 트렌드를 반영한 부스를 추가했다 매직 아일랜드에서는 언택트 트렌드에 맞춰 관객과 직접 접촉하는 콘텐츠를 줄이면서도 할로윈의 짜릿한 스릴을 느낄 수 있게 포토존과 공연 볼거리를 강화했다 메인브릿지에 좀비에게 습격받은 경찰차 좀비가 거꾸로 매달린 호러 공중전화박스 할로윈 분위기의 포토존을 설치했다 빅 대디와 좀비들이 매직캐슬을 점령하며 화려한 불기둥과 퍼포먼스를 선보이는 캐슬 좀비 M 공연은 매일 2회 열린다 1회 공연은 일몰시간에 맞춰 유동적으로 2회는 오후 8시에 막을 올린다 오후 8시30분부터는 3D 미디어 맵핑쇼 매직캐슬 라이츠업 할로윈 콘셉트로 업그레이드되어 공연한다 이밖에 워크스루형 호러 어트랙션 좀비프리즌 도 호러 영상관 좀비 씨어터 도 할로윈 시즌을 즐길 수 있는 우대도 다양하게 마련했다 23일까지 티몬 이베이 마이리얼트립을 통해 인당 3만4000원에 2회 입장이 가능한 종일권 재방문 패키지 판매한다 대인 1인과 1인이 입장 가능한 종일 가족권 재방문 패키지 도 5만8000원에 판매한다 삼성카드 또는 KT 

Input: 도널드 트럼프 미국 대통령은 기행과 막말 발언으로 구설수에 오른 인물이다 최근 미국 대선에서 패했지만 순순히 대통령 직에서 물러나지 않을 것으로 보인다 비판을 많이 받은 트럼프 대통령도 평가받을만한 점이 그는 역대 미국 대통령 중 유일하게 반려동물을 키우지 않은 대통령이었다 대통령 취임 이후 이미지 개선을 위해 반려견을 입양할 것을 권유했지만 트럼프 대통령은 거부했다 대통령 직무 수행만 해도 시간이 모자랄뿐더러 반려동물을 좋아하지 않는다는 이유에서다 반려동물을 좋아하지도 않으면서 이미지메이킹을 위해 반려견을 키우는 것은 기만행위라고 트럼프 대통령은 판단했다 쇼맨십과 기행으로 명성을 떨친 트럼프 대통령이지만 좋아하지 않기 키우지 않겠다 고 한 것은 그의 솔직한 면모를 보여줬다 자신이 싫어하더라도 정치적 이익을 위해 좋아하는 척할 수 있는 게 대부분의 정치인이 아닐까 동물권에 대한 논의가 이제 막 싹트기 시작한 한국에서는 반려동물을 액세서리로 보는 인식이 아직도 강하다 이미지메이킹을 위해 쓰다가 필요 없다고 판단되면 즉각 내다 버린다 포천파출소의 명예경찰견 왕방이와 왕순이가 전형적인 사례다 진도믹스견으로 유기견인 왕방이와 왕순이는 2018년과 2019년 강아지였을 파출소 경관들이 데리고 왔다 생후 2개월께 파출소에 온 두 마리의 개는 서로 의지하며 지냈다 명예경찰견으로 경찰과 시민들의 사랑을 담뿍 받았지만 지난 6월께 현실의 벽에 부딪쳤다 개들이 짖어서 불편하다는 동네 주민들의 민원이 접수되자 파출소 측은 개들을 곳에 보내기로 했다 개를 데려온 경관들도 근무지로 간 지 오래였다 왕방이와 왕순이는 갈 곳을 백방으로 알아봤지만 입양이 여의치 않았다 중형견 2마리를 단번에 입양하기는 쉽지 않기 때문이다 왕방이와 왕순이를 돌보던 이가 국외입양을 추진한 끝에 미국 캘리포니아로 입양을 떠났다 케이지에 갇혀 한국 땅을 떠나는 왕방이와 왕순이의 눈이 슬퍼 보였다는 소식이 전해지기도 했다 관리소홀로 어이없이 세상을 떠난 반려동물도 많다 유기묘였던 다행이는 2014년 

Input: 인천공항 지하에는 서울에서 대전까지보다 긴 130km 고속도로가 공항에서 맨 마지막으로 체크인하면 짐을 가장 먼저 받을 수 있을까 나폴레옹 철도의 폭이 달라졌다고 우리는 매일 길 위에서 다양한 교통수단을 이용해 생활하고 의식주 교통은 생활에서 떼려야 뗄 수 없는 존재이다 교통의 속사정은 잘 알지 못한다 모르면 단순한 이동수단일 뿐이지만 알면 똑똑하게 이용할 수 있는 정보 된다 인 저자는 수많은 자료와 해박한 지식으로 우리의 삶이자 일상인 교통에 대해 재미있고 쉽게 이야기를 풀어낸다 인천공항 지하에 130km 고속도로가 있는 이유는 무엇일까 국제 규격 축구장 53개를 합한 크기인 이곳에는 여행객의 수하물을 처리하는 BHS가 어마어마한 규모의 수하물 처리 시스템만 보아도 인천공항을 많은 사람이 이용하고 있는지 짐작할 수 여기에 더해 인천공항에서 수하물을 잘못 보내는 경우는 10만 개당 0 9개뿐이라는 사실 인천 미국 샌프란시스코 노선은 갈 때는 10시간 25분이 걸리는데 올 때는 13시간으로 2시간이 더 걸린다 반면 인천 런던 노선은 갈 때는 12시간 30분이 걸리지만 올 때는 11시간으로 시간이 짧아진다 이유는 무엇일까 제트기류 때문이다 제트기류를 만날 수 있는 가장 대표적인 노선은 인천 하와이 노선이다 하늘 길에도 병목 현상이 특정 구간에 항공편이 몰리면 비행기의 출발과 도착이 늦어질 수 있는 병목 현상이 발생한다 운항편이 집중되는 시간대라면 병목 현상도 심해진다 하늘 길도 도로처럼 새로운 길이 필요하다 발을 쭉 뻗어도 되는 비상구 좌석에는 앉을 수 있을까 맨 처음 체크인하면 짐을 제일 먼저 받을 수 있을까 인천공항을 살피는 9000개의 눈 승객은 모르는 벙크 비밀 위험한 흉기로 돌변하는 작은 새와 쇳조각 나라마다 항공료 세금 하늘 길과 관련된 재미있는 상식을 책에서 만날 수 첨단 KTX는 고운 모래를 싣고 다니는 것일까 기차는 바퀴와 선로 사이의 마찰력을 이용해 움직이는 구조다
Output: 앵 커   앵 커   앵 커   앵 커   앵 커   

Input: 박원순 서울시장이 실종됐다는 신고가 경찰에 접수됐다 9일 경찰에 따르면 이날 오후 5시께 박 시장이 연락이 두절됐다 는 신고가 들어왔다 경찰은 인력을 투입해 집 주변 일대를 수색 중이다 박 시장의 딸이 경찰에 실종신고를 했으며 현재 박 시장의 휴대전화는 꺼져있는 것으로 전해진다 이와 관련해 서울시는 9일 박원순 시장이 오늘 출근을 하지 않았다 며 실종설을 확인 중 이라고 밝혔다 이날 오전 10시 40분 박원순 시장의 오후 공개일정 취소 공지를 했다 서울경찰청은 청장 해당 사안과 관련해 청장 주재 대책회의를 하고 있는 것으로 알려졌다 동아닷컴
Output: 앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커
앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커

Input: 그것이 알고싶다 교양 프로그램 그것이 알고싶다 미국 테네시주 내슈빌 한인 감금 폭행 사건에 대해 추적해 큰 관심을 모았다 8일 밤 방송된 SBS TV 교양 프로그램 그것이 알고싶다 에서는 미국 테네시주 내슈빌에서 일어난 한인 감금 폭행 사건의 진실을 파헤쳤다 놀랍게도 논란의 인물은 그래미 어워즈 올해의 앨범상을 받은 케이시 머스그레이브스의 Golden Hour 제이슨 므라즈의 I m your s 쟁쟁한 팝스타들의 앨범에 참여한 한국인 음악 프로듀서 신 씨 신 씨는 지난 2018년 12월 미국 테네시주 내슈빌에서 자신의 여자친구를 잔혹하게 감금 후 폭행한 혐의로 체포됐다 현지 경찰에 따르면 신 씨는 약 7일간 본인의 집에 피해자를 감금해 청소기 드릴 머그잔 등으로 피해자를 구타하고 전깃줄로 목을 조르는 무차별적인 폭행을 가했다고 한다 신 씨는 피해자의 상처에 뜨거운 물을 붓거나 코에 물을 넣는 잔혹한 고문 행각도 벌였다고 알려졌다 이어 성고문까지 자행됐다고 전해졌다 피해자는 감금 7일째 되던 날 신 씨의 감시가 소홀해진 틈을 타 도망쳤고 이후 이웃 주민에게 구조돼 병원으로 옮겨졌다 신 씨는 보석금을 내고 풀려났고 이듬해 4월 신 씨가 또다시 피해자를 감금 폭행한 혐의로 체포되는 일이 발생했다 이와 관련해 신 씨의 누나는 동생이 유명해서 당한 일이라고 항변했다 피해자가 이상한 사람이라고 말해 논란을 일으켰다 동아닷컴
Output: 앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 커   앵 